In [1]:
import torch
from torch.optim import Adam
from torch.nn import Linear, MSELoss, Sequential, CrossEntropyLoss,Softmax
from torch.nn import BCELoss, Sigmoid, ReLU, Module, Conv2d, MaxPool2d
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.utils import to_categorical

In [2]:
(X_train, y_train), (X_test, y_test) = load_data()

In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
X_trainR = X_train.reshape(-1,1,28,28)
y_trainR = to_categorical(y_train)
x = torch.FloatTensor(X_trainR)
y = torch.FloatTensor(y_trainR)

C:\Users\NT551XCJ\AppData\Local\Temp\ipykernel_9792\3095431258.py:3: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  x = torch.FloatTensor(X_trainR)


In [5]:
class CNN( Module ):
    def __init__(self):
        super().__init__()

        self.conv1 = Conv2d( in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.conv2 = Conv2d( in_channels=16, out_channels=8, kernel_size=3, stride=1)
        self.max1 = MaxPool2d( kernel_size= 2)
        self.nn1 = Linear( 8*8*8,32)
        self.nn2 = Linear( 32,10)

    def forward( self, x):
        #print('연산전', x.size() ) #x.shape
        x = F.relu(  self.conv1(x) )
        #print('conv1 연산후', x.size() )
        x = F.relu(  self.conv2(x) )
        #print('conv2 연산후', x.size() )
        x = self.max1( x )
        #print('pool 연산후', x.size() )
        x = x.view( -1, 8*8*8) # x.reshape(-1, ??)
        x = F.relu( self.nn1(x) )
        x = F.softmax( self.nn2( x ), dim = 1 )
        return x

In [6]:
cnn = CNN()
cnn(x)

tensor([[5.5871e-08, 8.2167e-10, 5.9750e-06,  ..., 2.6047e-13, 4.9726e-10,
         4.1467e-02],
        [5.2945e-09, 6.5841e-02, 6.2919e-10,  ..., 3.3745e-15, 1.7918e-07,
         1.3265e-06],
        [1.5466e-04, 1.5762e-04, 9.5939e-09,  ..., 1.9963e-05, 4.7728e-03,
         6.2466e-01],
        ...,
        [5.7189e-03, 4.6490e-05, 4.0166e-06,  ..., 4.9972e-07, 3.1537e-07,
         1.1719e-02],
        [2.8393e-05, 7.0917e-04, 1.1201e-04,  ..., 2.5076e-06, 7.6052e-06,
         3.7167e-03],
        [4.7056e-04, 2.0619e-05, 1.9508e-05,  ..., 2.5883e-07, 1.0680e-02,
         3.1542e-02]], grad_fn=<SoftmaxBackward0>)

In [7]:
from sklearn.metrics import accuracy_score

In [ ]:
model = CNN()
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 0.01)

for step in range(10):
    optimizer.zero_grad()
    hx = model(x)
    cost = loss_fn(hx, y)
    cost.backward()
    optimizer.step()
    print('=========================')
    realData = y.argmax(dim = 1).numpy()
    pred = hx.argmax(dim = 1).numpy()
    print('정확도 :', accuracy_score(realData, pred))
    print('step :', step, cost.item())